# Environment Setup, Download dataset and model

In [1]:
!pip install gdown
!pip install sentence-transformers


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#this download the dataset from my drive - which is already converted into .csv
!gdown 1XYC0lLXq1kmdkbEYD1fOrIG990rpVgiZ
#this download the model from my drive - which I had trained for 100 epoch
!gdown https://drive.google.com/drive/folders/1-2dzlOGVlo5DIrVcFJOA49Tnco7sdaEd --folder

Downloading...
From: https://drive.google.com/uc?id=1XYC0lLXq1kmdkbEYD1fOrIG990rpVgiZ
To: d:\Projects\sentence-transformers\examples\training\ONet_Job_Title_Embeddings\Alternate Titles.csv

  0%|          | 0.00/4.31M [00:00<?, ?B/s]
 12%|█▏        | 524k/4.31M [00:01<00:07, 483kB/s]
 24%|██▍       | 1.05M/4.31M [00:02<00:06, 529kB/s]
 36%|███▋      | 1.57M/4.31M [00:03<00:05, 527kB/s]
 49%|████▊     | 2.10M/4.31M [00:03<00:04, 530kB/s]
 61%|██████    | 2.62M/4.31M [00:04<00:03, 533kB/s]
 73%|███████▎  | 3.15M/4.31M [00:06<00:02, 519kB/s]
 85%|████████▌ | 3.67M/4.31M [00:06<00:01, 546kB/s]
 97%|█████████▋| 4.19M/4.31M [00:07<00:00, 537kB/s]
100%|██████████| 4.31M/4.31M [00:08<00:00, 547kB/s]
100%|██████████| 4.31M/4.31M [00:08<00:00, 534kB/s]


Retrieving folder 1-nRnBzmz8iJEQf9d6I1gR_0bulD9wenb 1_Pooling
Processing file 10JQsjpyG6sCfeJR_rKJiUQIXNzB6Leg1 config.json
Retrieving folder 1-j_djypmsF8GOJ2t_xDRfcmurkc0Ca21 2_Normalize
Processing file 1-SFba0wy3ZwyIgqg5zZU60hvb7aIivtG config_sentence_transformers.json
Processing file 1-Pucz61VNarBP9B6N4fUX-5L9Q_U7Ao3 config.json
Retrieving folder 1-EEWqyZIPVPnvCkWkiJtAOgG8Chb0vOH eval
Retrieving folder 1wL6ssL66uBRc_NBgSBHKcGGNEkZbuRr2 .ipynb_checkpoints
Processing file 1-2A_eIcCtQhmHvflnCdcyDba_1ah_y2O triplet_evaluation_TrainTriplet_results.csv
Processing file 1-g5AM0sKLgtp7WyCaqFq0PqofnrDgQvq modules.json
Processing file 1-O2Vvn0BFh6iwzqVvldQaAa5RK4hQZbf pytorch_model.bin
Processing file 1-UbQjSoTMEJSTWAf-gqilf4FjOlupTUs README.md
Processing file 1-qxvj06iQzYnIvQMxO9CkT99QTga_awi sentence_bert_config.json
Processing file 10Cyfm6p6grRJI3h_gfL_gAi3xSPduGGs special_tokens_map.json
Processing file 10FV4fbzUsB-x6LYgd7NeM8Qxg-mpqS-W tokenizer_config.json
Processing file 1-xhdKRbODPyarL

Retrieving folder list
Retrieving folder list completed
Building directory structure
Downloading...
From: https://drive.google.com/uc?id=10JQsjpyG6sCfeJR_rKJiUQIXNzB6Leg1
To: d:\Projects\sentence-transformers\examples\training\ONet_Job_Title_Embeddings\all-MiniLM-L6-ONet-Titles\1_Pooling\config.json

  0%|          | 0.00/190 [00:00<?, ?B/s]
100%|██████████| 190/190 [00:00<?, ?B/s] 
Downloading...
From: https://drive.google.com/uc?id=1-SFba0wy3ZwyIgqg5zZU60hvb7aIivtG
To: d:\Projects\sentence-transformers\examples\training\ONet_Job_Title_Embeddings\all-MiniLM-L6-ONet-Titles\config_sentence_transformers.json

  0%|          | 0.00/116 [00:00<?, ?B/s]
100%|██████████| 116/116 [00:00<00:00, 58.8kB/s]
Downloading...
From: https://drive.google.com/uc?id=1-Pucz61VNarBP9B6N4fUX-5L9Q_U7Ao3
To: d:\Projects\sentence-transformers\examples\training\ONet_Job_Title_Embeddings\all-MiniLM-L6-ONet-Titles\config.json

  0%|          | 0.00/687 [00:00<?, ?B/s]
100%|██████████| 687/687 [00:00<00:00, 691kB/

# Utility Function

In [3]:
from torch.nn import CosineSimilarity
from collections import defaultdict
import random

def computeEmbCosineSim(model, df):
  """
  This function the average embedded similarities between 'Alternate Title' and 'Title' using an SBert model.
  :param model: Sentence Transformer model
  :param df: a pandas dataframe of ONet's 'Alternative Titles.csv' 
  :return:
    titleList: a list of titles
    embSimList: a list of average embedding similarities between 'Title' and 'Alternate Title'
  """
  groupedDf = df.groupby(['Title'])
  embSimList = []
  titleList = []
  for groupName, group in groupedDf:
    titleVec = model.encode(groupName, convert_to_tensor = True).unsqueeze(dim=0)
    alternateTitleMatrix = model.encode(group['Alternate Title'].tolist(), convert_to_tensor = True)
    CosSim = CosineSimilarity(dim =1, eps = 1e-08)
    simVec = CosSim(titleVec, alternateTitleMatrix)
    embSimList.append(simVec.mean())
    titleList.append(groupName)
  return titleList, embSimList

def triplets_from_labeled_dataset(input_examples):
  """
  This function create a list of triplet for a [(label, sentence), (label, sentence)...] dataset.
  by using each example as an anchor and then:
    - randomly select a positive instance (with the same label)
    - randomly select a negative instance (with different label)
  :param input_examples: [InputExample] a list of labeled InputExample
  :return:
    triplets: [InputExample] where InputExample.texts=[anchor, positive, negative]
  """
  triplets = []
  label2sentence = defaultdict(list)
  for inp_example in input_examples:
      label2sentence[inp_example.label].append(inp_example)

  for inp_example in input_examples:
      anchor = inp_example
      if len(label2sentence[inp_example.label]) < 2: #We need at least 2 examples per label to create a triplet
          continue

      positive = None
      while positive is None or positive == anchor:
          positive = random.choice(label2sentence[inp_example.label])

      negative = None
      while negative is None or negative.label == anchor.label:
          negative = random.choice(input_examples)

      triplets.append(InputExample(texts=[anchor.texts[0], positive.texts[0], negative.texts[0]]))

  return triplets

c:\Users\Admin\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Exploratory Data Analysis + Test Pretrained model

In [4]:
import pandas as pd

df = pd.read_csv(r'Alternate Titles.csv')
df.head()

,O*NET-SOC Code,Title,Alternate Title,Short Title,Source(s)
0,11-1011.00,Chief Executives,Aeronautics Commission Director,NaN,8
1,11-1011.00,Chief Executives,Agricultural Services Director,NaN,8
2,11-1011.00,Chief Executives,Arts and Humanities Council Director,NaN,8
3,11-1011.00,Chief Executives,Bank President,NaN,9
4,11-1011.00,Chief Executives,Bureau Chief,NaN,"04,06"


In [5]:
df.nunique()

O*NET-SOC Code      1016
Title               1016
Alternate Title    42889
Short Title         1733
Source(s)             97
dtype: int64

Problem Definition: There are various titles that meant the same occupation. The actual occupation is stored within `Title`, whereas titles refering to the same occupation is stored withn `Alternate Title`.

Our job is to train a sentence embedding that:
- minimize euclidian distance between titles refering to the same occupation.
- maximize euclidian distance between titles refering to different occupation.

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

titleList, embSimList = computeEmbCosineSim(model, df)
minSim = 1
maxSim = -1
maxName = ''
minName = ''

avgSim = 0
for name, averageSim in zip(titleList, embSimList):
  avgSim += averageSim
  print("{}: {:.4f}".format(name, averageSim))
  if averageSim > maxSim:
    maxSim = averageSim
    maxName = name
  if averageSim < minSim:
    minSim = averageSim
    minName = name
avgSim = avgSim / len(embSimList)
print('\n-----------------------------------------------------------\n')
print("Average similarity across all occupation is: {:.4f}".format(avgSim))
print("Mininal Similarity is: {} - {:.4f}".format(minName, minSim))
print("Maximal Similarity is: {} - {:.4f}".format(maxName, maxSim))

Accountants and Auditors: 0.6168
Actors: 0.4465
Actuaries: 0.5065
Acupuncturists: 0.5345
Acute Care Nurses: 0.6114
Adapted Physical Education Specialists: 0.7437
Adhesive Bonding Machine Operators and Tenders: 0.3239
Administrative Law Judges, Adjudicators, and Hearing Officers: 0.5211
Administrative Services Managers: 0.6917
Adult Basic Education, Adult Secondary Education, and English as a Second Language Instructors: 0.5151
Advanced Practice Psychiatric Nurses: 0.7199
Advertising Sales Agents: 0.5112
Advertising and Promotions Managers: 0.5564
Aerospace Engineering and Operations Technologists and Technicians: 0.5794
Aerospace Engineers: 0.6147
Agents and Business Managers of Artists, Performers, and Athletes: 0.5124
Agricultural Engineers: 0.6882
Agricultural Equipment Operators: 0.3990
Agricultural Inspectors: 0.5259
Agricultural Sciences Teachers, Postsecondary: 0.5674
Agricultural Technicians: 0.5291
Agricultural Workers, All Other: 0.1957
Air Crew Members: 0.4494
Air Crew Offic

Pretrained model seem to see that `Alternative Title` and `Title` doesnt have much similarity.

In [7]:
import numpy as np
model = SentenceTransformer('all-MiniLM-L6-v2')

sameTitleSentences = ['Market Development Executive', 
                      'Brand Manager',
                      'Marketing Manager', 
                      'Sales and Marketing Manager', 
                      'Sales Marketing Director']
sameTitleEmbeddings = model.encode(sameTitleSentences)

diffTitleSentences = ['Chief Passenger Ship Steward', 
                      'Chief Technology Officer',
                      'Security Engineer', 
                      'Data Engineer', 
                      'Business Development Director']

diffTitleEmbeddings = model.encode(diffTitleSentences)
# i know this isn't the same as Cosine Similarity, but it still get the job done (as embedding already normalized to 1 lol)
sameTitleCorrMat = np.einsum('id, jd -> ij', sameTitleEmbeddings, sameTitleEmbeddings)
diffTitleCorrMat = np.einsum('id, jd -> ij', diffTitleEmbeddings, diffTitleEmbeddings)

print('Correlation matrix of titles refering to the same occupation \n',sameTitleCorrMat)
print('\n Correlation matrix of titles refering to diffrent occupation \n',diffTitleCorrMat)

Correlation matrix of titles refering to the same occupation 
 [[0.9999999  0.4354058  0.60128295 0.5684756  0.5932191 ]
 [0.4354058  0.99999994 0.73901844 0.6674547  0.5470799 ]
 [0.60128295 0.73901844 1.         0.90519404 0.7384229 ]
 [0.5684756  0.6674547  0.90519404 1.         0.8052912 ]
 [0.5932191  0.5470799  0.7384229  0.8052912  1.        ]]

 Correlation matrix of titles refering to diffrent occupation 
 [[1.         0.39152807 0.19768119 0.13304308 0.13692378]
 [0.39152807 1.         0.49181604 0.45678002 0.4399817 ]
 [0.19768119 0.49181604 1.         0.67144156 0.34228265]
 [0.13304308 0.45678002 0.67144156 1.0000002  0.37102643]
 [0.13692378 0.4399817  0.34228265 0.37102643 1.        ]]


# Transfer Learning using SBert on ONET dataset

In [10]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers.datasets import SentenceLabelDataset
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import TripletEvaluator
import random

from torch.utils.data import DataLoader

In [11]:
#Create a dataset from our pandas dataframe
groupedDict = df.groupby(['Title'])['Alternate Title'].apply(list).to_dict()

dataset = [] #list of InputExample; Where: texts: an alternative title ; label: an integer representing that label 
for i, key in enumerate(groupedDict.keys()):
    for v in groupedDict[key]:
        dataset.append(
            InputExample(texts=[v], 
                        label=i))
#Create evaluation triplets set (this is train set tbh)
trainTriplets = triplets_from_labeled_dataset(dataset) 

In [ ]:
outPath = 'all-MiniLM-L6-ONet-Titles'
modelPath = 'all-MiniLM-L6-ONet-Titles'

# Create SentenceLabelDataset, DataLoader, Loss and Evaluator
trainLabelSet = SentenceLabelDataset(dataset, 2)
train_dataloader = DataLoader(trainLabelSet, batch_size=512) #batch size is dependent on your GPU RAM

train_loss = losses.BatchAllTripletLoss(model=model, margin = 0.3167) 
#default magin=5 lead to model collapse

trainEvaluator = TripletEvaluator.from_input_examples(trainTriplets, name='TrainTriplet')

#model = SentenceTransformer('all-MiniLM-L6-v2') # train from pretrained
model = SentenceTransformer(modelPath) #train from your saved model

model.fit(train_objectives=[(train_dataloader, train_loss)],epochs=100,
          evaluator = trainEvaluator, output_path = outPath)

In [12]:
modelPath = 'all-MiniLM-L6-ONet-Titles'
model = SentenceTransformer(modelPath)

titleList, embSimList = computeEmbCosineSim(model, df)
minSim = 1
maxSim = -1
maxName = ''
minName = ''

avgSim = 0
for name, averageSim in zip(titleList, embSimList):
  avgSim += averageSim
  print("{}: {:.4f}".format(name, averageSim))
  if averageSim > maxSim:
    maxSim = averageSim
    maxName = name
  if averageSim < minSim:
    minSim = averageSim
    minName = name
avgSim = avgSim / len(embSimList)
print('\n-----------------------------------------------------------\n')
print("Average similarity across all occupation is: {:.4f}".format(avgSim))
print("Mininal Similarity is: {} - {:.4f}".format(minName, minSim))
print("Maximal Similarity is: {} - {:.4f}".format(maxName, maxSim))

Accountants and Auditors: 0.8512
Actors: 0.9010
Actuaries: 0.6533
Acupuncturists: 0.9349
Acute Care Nurses: 0.8656
Adapted Physical Education Specialists: 0.9486
Adhesive Bonding Machine Operators and Tenders: 0.8478
Administrative Law Judges, Adjudicators, and Hearing Officers: 0.9006
Administrative Services Managers: 0.9227
Adult Basic Education, Adult Secondary Education, and English as a Second Language Instructors: 0.9066
Advanced Practice Psychiatric Nurses: 0.9090
Advertising Sales Agents: 0.8752
Advertising and Promotions Managers: 0.8927
Aerospace Engineering and Operations Technologists and Technicians: 0.8115
Aerospace Engineers: 0.8407
Agents and Business Managers of Artists, Performers, and Athletes: 0.8476
Agricultural Engineers: 0.8574
Agricultural Equipment Operators: 0.7780
Agricultural Inspectors: 0.8610
Agricultural Sciences Teachers, Postsecondary: 0.8363
Agricultural Technicians: 0.7553
Agricultural Workers, All Other: 0.7820
Air Crew Members: 0.8663
Air Crew Offic

The similarities between `Title` and `Alternate Title` shot up for a majority of occupations.

There are some odd cases, such as `Life, Physical, and Social Science Technicians, All Other`

In [13]:
import numpy as np

modelPath = 'all-MiniLM-L6-ONet-Titles'
model = SentenceTransformer(modelPath)

sameTitleSentences = ['Market Development Executive', 
                      'Brand Manager',
                      'Marketing Manager', 
                      'Sales and Marketing Manager', 
                      'Sales Marketing Director']
sameTitleEmbeddings = model.encode(sameTitleSentences)

diffTitleSentences = ['Chief Passenger Ship Steward', 
                      'Chief Technology Officer',
                      'Security Engineer', 
                      'Data Engineer', 
                      'Business Development Director']                      
diffTitleEmbeddings = model.encode(diffTitleSentences)

sameTitleCorrMat = np.einsum('id, jd -> ij', sameTitleEmbeddings, sameTitleEmbeddings)
diffTitleCorrMat = np.einsum('id, jd -> ij', diffTitleEmbeddings, diffTitleEmbeddings)

print('Correlation matrix of titles refering to the same occupation \n',sameTitleCorrMat)
print('\n Correlation matrix of titles refering to diffrent occupation \n',diffTitleCorrMat)

Correlation matrix of titles refering to the same occupation 
 [[0.9999999  0.9144856  0.96051633 0.96492237 0.9541205 ]
 [0.9144856  0.99999994 0.94745225 0.95761406 0.95256233]
 [0.96051633 0.94745225 0.9999999  0.9859412  0.98536265]
 [0.96492237 0.95761406 0.9859412  1.0000002  0.99032617]
 [0.9541205  0.95256233 0.98536265 0.99032617 1.0000002 ]]

 Correlation matrix of titles refering to diffrent occupation 
 [[ 1.          0.40324318 -0.14017741 -0.1226861   0.23255512]
 [ 0.40324318  1.0000001  -0.10699292  0.15405846  0.79047686]
 [-0.14017741 -0.10699292  0.9999999   0.2778319  -0.2330278 ]
 [-0.1226861   0.15405846  0.2778319   1.0000002   0.04461809]
 [ 0.23255512  0.79047686 -0.2330278   0.04461809  0.99999994]]


In [20]:
import numpy as np

modelPath = 'all-MiniLM-L6-ONet-Titles'
model = SentenceTransformer(modelPath)

# It even works for data not found in the ONet database
sameTitleSentences = ['Help Desk Technician', 
                      'IT Help Desk',
                      'IT Help Desk Support', 
                      'Helpdesk Technician', 
                      'Jr. Helpdesk/IT Support']
sameTitleEmbeddings = model.encode(sameTitleSentences)

diffTitleSentences = ['Scrum Master', 
                      'Helpdesk Technician',
                      'IT Security Specialist', 
                      'IT Project Manager', 
                      'Documentation Specialist']                      
diffTitleEmbeddings = model.encode(diffTitleSentences)

sameTitleCorrMat = np.einsum('id, jd -> ij', sameTitleEmbeddings, sameTitleEmbeddings)
diffTitleCorrMat = np.einsum('id, jd -> ij', diffTitleEmbeddings, diffTitleEmbeddings)

print('Correlation matrix of titles refering to the same occupation \n',sameTitleCorrMat)
print('\n Correlation matrix of titles refering to diffrent occupation \n',diffTitleCorrMat)

Correlation matrix of titles refering to the same occupation 
 [[0.99999994 0.9093549  0.97199523 0.8820429  0.7670768 ]
 [0.9093549  0.99999994 0.9482164  0.8774843  0.8122248 ]
 [0.97199523 0.9482164  0.9999999  0.8880424  0.8158312 ]
 [0.8820429  0.8774843  0.8880424  1.0000001  0.7891536 ]
 [0.7670768  0.8122248  0.8158312  0.7891536  1.        ]]

 Correlation matrix of titles refering to diffrent occupation 
 [[ 1.          0.31943572  0.3078606   0.8682898   0.06573086]
 [ 0.31943572  0.9999999   0.41069573  0.29315668  0.40438908]
 [ 0.3078606   0.41069573  1.          0.28673965 -0.05115794]
 [ 0.8682898   0.29315668  0.28673965  1.0000002   0.08014496]
 [ 0.06573086  0.40438908 -0.05115794  0.08014496  1.        ]]


**Notes:** It even understand that `Scrum Master` and `IT Project Manager` is similar to one another, compared to engineers jobs.